# Numpy

```
conda install numpy
```

Let's start with a plausible problem. We have a dataset of all daily temperatures measured at Newark since 1893 and we want to analyze it. First, let's try that with a Python list.

In [ ]:
with open("data/newark-temperature-avg.txt") as file:
    temperatures = [float(line) for line in file]

len(temperatures), temperatures[:10], temperatures[-10:]

Much of the record is missing, as we can see by counting NaNs:

In [ ]:
import math

# Let's misuse the fact that True is 1, False is 0
numbad = sum(math.isnan(x) for x in temperatures)

print(f"Fraction of values that are NaN: {numbad / len(temperatures):.2%}")

We have a more complete dataset of daily minimum and maximum temperatures. It's not as accurate, but we can impute the missing averages by averaging the minimum and maximum.

In [ ]:
with open("data/newark-temperature-min.txt") as file:
    min_temperatures = [float(line) for line in file]


with open("data/newark-temperature-max.txt") as file:
    max_temperatures = [float(line) for line in file]

(len(min_temperatures), min_temperatures[:10], min_temperatures[-10:],
 len(max_temperatures), max_temperatures[:10], max_temperatures[-10:])

While we fill in the missing values, let's also measure how long it takes.

In [ ]:
%%timeit

imputed_temperatures = []
for average, minimum, maximum in zip(temperatures, min_temperatures, max_temperatures):
    if math.isnan(average):
        imputed_temperatures.append(0.5 * (minimum + maximum))
    else:
        imputed_temperatures.append(average)

Now let's do the same thing in Numpy, again measuring the time.

In [ ]:
import numpy as np

arr_temperatures = np.array(temperatures)
amin_temperatures = np.array(min_temperatures)
amax_temperatures = np.array(max_temperatures)

In [ ]:
%%timeit

missing = np.isnan(temperatures)
imputed_temperatures = arr_temperatures.copy()
imputed_temperatures[missing] = 0.5 * (amin_temperatures[missing] + amax_temperatures[missing])

Or just

In [ ]:
%%timeit

imputed_temperatures = np.where(
    np.isnan(arr_temperatures),                      # condition
    0.5 * (amin_temperatures + amax_temperatures),   # if true
    arr_temperatures                                 # if false
)

We see that Numpy can be much faster than Python loops, in this case a factor of 100 or 200, but it can be as high as several thousand. (It depends on the application.) The way you tell it what to do is also very different, which may be good or bad. It may read more naturally, maybe not.

### Differences

Numpy arrays have a single data type, or `dtype`, unlike Python lists.

In [ ]:
np.zeros(5, dtype=np.float64)

In [ ]:
[0, 0.0, "0"]

This is where a large part of Numpy's speed comes from. When Python churns, a lot of that time is spent checking and re-checking data types, which in a compiled language like C++ were checked once and for all in the compilation step.

In [ ]:
%%timeit

imputed_temperatures = []
for average, minimum, maximum in zip(arr_temperatures, amin_temperatures, amax_temperatures):
    if math.isnan(average):
        imputed_temperatures.append(0.5 * (minimum + maximum))
    else:
        imputed_temperatures.append(average)

When you write a loop, Python checks the type each time. It does a variety of other things too, each time it runs a line of code. The inner portion of the numpy function does none of this, but just runs (mostly) machine level instructions on each item directly performing the calculation.

Python was also building a list, whereas Numpy could prepare the output array directly from the input data sizes.

Numpy is a suite of compiled functions applied to data with predetermined types. When you're using Numpy properly, you'll have very few `for` loops and `if` statements in your code: the Python code acts as a high-level director, while Numpy does its looping in compiled code.

The Numpy library consists mainly of one class, `np.ndarray`, and operations on it. This is an n-dimensional array of contiguous data. Some operations change that data or make new arrays, but many operations merely change our interpretation of the data. Those that leave the data alone are fastest.

In [ ]:
array = np.arange(24, dtype=np.float64)    # 64-bit floating point numbers
array

In [ ]:
array.view(np.int64)

In [ ]:
array.tostring()

In [ ]:
array = np.arange(24, dtype=np.float64)    # 64-bit floating point numbers
array

In [ ]:
array.reshape(6, 4)

In [ ]:
array.reshape(6, 4, order="f")    # Fortran order vs C order: how a 1D sequence covers an nD block

This interpretation has only two parameters:

   * `dtype` (data type, including endianness): how bytes are represented as numbers
   * `shape` and `order`: how those numbers are arranged in an n-dimensional grid

Mistakes in interpretation are usually not subtle, so just be sure to _look_ at your data.

Numpy arrays can be used in mathematical formulae, but instead of computing one value, they compute a whole array of values, element by element.

In [ ]:
a = np.arange(10)
a

In [ ]:
a + 100

In [ ]:
b = np.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [ ]:
a + b

In [ ]:
a**2

Generally, you can imagine a table of data to compute: the columns represent meaningful quantities (often named) while the rows represent anonymous instances.

In [ ]:
a = np.random.uniform(5, 10, 10000)
b = np.random.uniform(10, 20, 10000)
c = np.random.uniform(-0.1, 0.1, 10000)
len(a)

A conventional Python approach would be to compute the formula on each instance, one after another.

In [ ]:
roots1 = []
for ai, bi, ci in zip(a, b, c):
    roots1.append((-bi + math.sqrt(bi**2 - 4*ai*ci)) / (2*ai))

The Numpy approach computes each step of the formula on all instances before moving on to the next step.

In [ ]:
roots2 = (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

The Numpy expression (`(-b + np.sqrt(b**2 - 4*a*c)) / (2*a)`) is equivalent to:

In [ ]:
tmp1 = np.negative(b)            # -b
tmp2 = np.square(b)              # b**2
tmp3 = np.multiply(4, a)         # 4*a
tmp4 = np.multiply(tmp3, c)      # tmp3*c
tmp5 = np.subtract(tmp2, tmp4)   # tmp2 - tmp4
tmp6 = np.sqrt(tmp5)             # sqrt(tmp5)
tmp7 = np.add(tmp1, tmp6)        # tmp1 + tmp6
tmp8 = np.multiply(2, a)         # 2*a
roots3 = np.divide(tmp7, tmp8)   # tmp7 / tmp8

One strange (but useful!) consequence of this rule that mathematical operations are applied elementwise is that it even applies to comparisons. Suppose we want to verify that the `roots1` computed in the Python loop match the `roots2` and `roots3` computed by Numpy.

In [ ]:
roots1 == roots2

In [ ]:
roots2 == roots3

When you want to check that _all_ of the elements are equal, use `np.all()` (or `np.any()` for at least one).

In [ ]:
np.all(roots2 == roots3)

In [ ]:
np.all(roots1 == roots2)

Why is that? Didn't we just see that `roots1 == roots2` is `True, True, True, ...`?

In [ ]:
np.any(roots1 == roots2)

In [ ]:
np.sum(roots1 == roots2), len(roots1)

Which ones fail?

In [ ]:
failures, = np.nonzero(roots1 != roots2)
failures

> #### Sneaky syntax
>
> Notice the comma above. Do you know what it does?

In [ ]:
roots1[failures[0]], roots2[failures[0]]

In [ ]:
roots1[failures[0]] - roots2[failures[0]]

Numpy uses different routines to do its calculations, so results might not be exactly the same. We don't care about last-digit differences, so we set a tolerance.

In [ ]:
np.all(abs(roots1 - roots2) < 1e-15)

In fact, this is so common there's a numpy function for this:

In [ ]:
np.allclose(roots1, roots2)

> ### Which is better?
>
> While opinions differ, in most cases, the hand calculation describes *how* to compute something, while the specific routine (np.allclose) conveys meaning about *why* the computation was made. The latter is an abstraction that scales much better than "how".

The upshot of this is that you can perform a calculation with the same expression on Numpy arrays as on Python scalars, as long as it's being applied to a table of numbers (i.e. arrays of all the same length).

Now let's get into some fancier gymnastics.

Python has a wonderfully consistent syntax for _slicing_ lists (or tuples or whatever):

In [ ]:
alist = [0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]
alist[4]

In [ ]:
alist[4:]

In [ ]:
alist[:7]

In [ ]:
alist[-1]

In [ ]:
alist[:-3]

In [ ]:
alist[3:8]

The third argument of a slice is the _stride,_ the amount to skip between elements.

In [ ]:
alist = [0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]
alist[3:8:2]

In [ ]:
alist[::2]

In [ ]:
alist[::-2]

In [ ]:
alist[5::-1]

Numpy arrays share this syntax— everything that works for a list works for an array— but they extend it considerably.

In [ ]:
array = np.array([[0.0, 1.1, 2.2, 3.3], [0, 10.1, 20.2, 30.3], [0, 100.1, 200.2, 300.3]])
array

In [ ]:
array[1]

In [ ]:
array[:, 1]

In [ ]:
array[1:, 2:]

In [ ]:
array[::2, 1::2]

Even arrays (or sequences) of booleans or integers can be slices.

In [ ]:
array = np.array([[0.0, 1.1, 2.2, 3.3], [0, 10.1, 20.2, 30.3], [0, 100.1, 200.2, 300.3]])
array[[False, True, True]]

In [ ]:
array[[2, 1, 0]]

In [ ]:
array[[2, 1, 1, 1, 1, 0, 0, 2]]

What could this possibly be useful for?

**Masking:** selecting elements, compacting a list

In [ ]:
a = np.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
b = np.array([0, 100, 200, 300, 400, 500, 600, 700, 800, 900])

In [ ]:
b > 400

In [ ]:
a[b > 400]

**Sorting and maximizing:**

In [ ]:
a = np.random.normal(0, 5, 50)
b = abs(a)

In [ ]:
np.argsort(a)

In [ ]:
a[np.argsort(a)]

In [ ]:
a = np.meshgrid(np.linspace(-5, 5, 11), np.linspace(-5, 5, 11))[1]
a

In [ ]:
a.argmax(axis=0)

In [ ]:
a[a.argmax(axis=0)]

**Dictionary encoding:**

In [ ]:
text = """Four score and seven years ago our fathers brought forth on this
continent, a new nation, conceived in Liberty, and dedicated to the proposition
that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any
nation so conceived and so dedicated, can long endure. We are met on a great
battle-field of that war. We have come to dedicate a portion of that field, as
a final resting place for those who here gave their lives that that nation might
live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate -- we can not consecrate -- we can
not hallow -- this ground. The brave men, living and dead, who struggled here,
have consecrated it, far above our poor power to add or detract. The world will
little note, nor long remember what we say here, but it can never forget what
they did here. It is for us the living, rather, to be dedicated here to the
unfinished work which they who fought here have thus far so nobly advanced. It
is rather for us to be here dedicated to the great task remaining before us --
that from these honored dead we take increased devotion to that cause for which
they gave the last full measure of devotion -- that we here highly resolve that
these dead shall not have died in vain -- that this nation, under God, shall
have a new birth of freedom -- and that government of the people, by the people,
for the people, shall not perish from the earth."""

In [ ]:
words = text.replace(".", "").replace(",", "").replace("--", "").split()

In [ ]:
dictionary, integers = np.unique(words, return_inverse=True)

In [ ]:
len(words), len(dictionary)

In [ ]:
dictionary

In [ ]:
integers

In [ ]:
dictionary[integers]

Notice that slicing with integer indexes is a function in the mathematical sense. An array is a mapping from integers to the values of the array: `[0, N) → V`.

Slicing with integer indexes composes a function `[0, m) → [0, N)` with it to get a new function `[0, m) → V`.

In [ ]:
a = np.array(["zero", "one", "two", "three", "four"])
b = np.array([3, 3, 1, 2, 4, 0, 1])

a[b]

Putting things together: we can use reshaping and fancy indexing together to do some surprisingly powerful things. ("Look, ma! No for loops!")

For example, I once had to reverse a list in groups of 8. I struggled with it until I found this online:

In [ ]:
original = np.array([7, 6, 5, 4, 3, 2, 1, 0,
                        15, 14, 13, 12, 11, 10, 9, 8,
                        23, 22, 21, 20, 19, 18, 17, 16,
                        31, 30, 29, 28, 27, 26, 25, 24])

In [ ]:
original.reshape(4, 8)[:, ::-1].reshape(32)

**Question:** How would we change this if we wanted to turn

In [ ]:
original = np.array([7, 6, 5, 4, 3, 2, 1, 0,
                        15, 14, 13, 12, 11, 10, 9, 8,
                        23, 22, 21, 20, 19, 18, 17, 16,
                        31, 30, 29, 28, 27, 26, 25, 24])

into [31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]?

In [ ]:
original.reshape(4, 8)[:, ::-1].reshape(32)    # change this line!

Also unlike Python slices, you can even _assign_ to sliced Numpy arrays. This overwrites parts of the array.

In [ ]:
array = np.arange(10) * 1.1
array

In [ ]:
array[5:] = -9
array

In [ ]:
array[::2] = 123
array

In [ ]:
array = np.ones(10, dtype=np.int64) * 999
array[np.array([False, False, False, True, True, False, True, False, True, False])] = np.array([1, 2, 3, 4])
array

In [ ]:
array = np.ones(10, dtype=np.int64) * 999
array[np.array([8, 7, 3, 0])] = np.array([1, 2, 3, 4])
array

**Example:** Rubik's square. Let's use the `roll` function to rotate rows and columns of a square matrix.

In [ ]:
rubiks = (np.tile(np.arange(10) * 0.1, 10) + np.repeat(np.arange(10), 10)).reshape(10, 10)
rubiks

In [ ]:
def twist(isrow, index, howmuch):
    if isrow:
        rubiks[index, :] = np.roll(rubiks[index, :], howmuch)
    else:
        rubiks[:, index] = np.roll(rubiks[:, index], howmuch)
    return rubiks

In [ ]:
twist(True, 3, 5)

**Exercise:** Shuffling cards. Suppose you want to model the first cut of a deck of cards. The dealer roughly cuts the deck in two and flips them together such that all cards from the left hand are in order relative to each other and all cards from the right hand are in order relative to each other, but they are randomly interleaved.

<center><img src="img/cards-chance-deck-19060.jpg" width="25%" /></center>

In [ ]:
cards = np.arange(52)
random_booleans = np.random.randint(0, 2, 52, dtype=np.bool)
negated_booleans = ~random_booleans
solution = cards.copy()    # overwrite this array with your solution!

In [ ]:
???

**Something to be careful about:** copies vs views. Numpy tries to operate as little as possible on the big datasets, and that means that it sometimes returns "an array" that is just a reinterpretation of the data. The old and new array objects both point to the same bytes in memory. Example:

In [ ]:
a = np.arange(10) * 1.1
a

In [ ]:
b = a[::3]
b

In [ ]:
b[:] = 999
b

In [ ]:
a

Changing `b` changes `a`. This can cause some _terrible_ bugs.

... but only upon assignment. If you mostly just apply functions to arrays, Numpy will share memory among stages of your calculation and run as efficiently as possible. When you do need to make an assignment, be careful to check your work (on an interactive prompt or in a notebook).

Here's a way to check:

In [ ]:
b.flags.owndata

In [ ]:
b.base is a

And here's a way to ensure that you get a copy with no link to the original:

In [ ]:
b = a[::3].copy()

**Tricky walk-through exercise:** Let's make a PNG! PNG files have a relatively simple structure ([full specification](https://www.w3.org/TR/PNG)), simple enough that we can make one without using a specialized library.

The format consists of a fixed preamble and compressed data blocks. The purpose of this exercise is to solve array restructuring problems that are similar to what you might encounter in a data analysis— half the problem is data-munging, right?— and end up with a cool tool for visualizing what various functions do in the remainder of this session.

The first part is easy: a preamble consisting of the following bytes: `b"\x89PNG\r\n\x1a\n"`. Let's write a function that produces this as a Numpy array.

In [ ]:
def png_preamble():
    return np.frombuffer(b"\x89PNG\r\n\x1a\n", dtype="u1")    # u1 is unsigned 1-byte integers: raw bytes

png_preamble()

Next are three "chunks." A "chunk" is a four-byte "tag," some data, and a "cyclic redundancy check" (CRC), a way of cross-checking that the data haven't been garbled. Let's write a function that constructs a chunk, using Numpy to reinterpret numbers as bytes (changing `dtype`).

<center><img src="img/png-spec-chunks.png" width="50%"></center>

In [ ]:
import zlib

def png_chunk(tag, data):
    out = np.empty(4 + len(tag) + len(data) + 4, dtype="u1")
    
    length_as_u4 = np.array([len(data)], dtype=">u4")    # 4-byte integer with most significant byte first (">")
    out[0:4] = length_as_u4.view("u1")
    
    out[4:8] = np.frombuffer(tag, dtype="u1")
    out[8 : 8 + len(data)] = np.frombuffer(data, dtype="u1")
    
    crc = zlib.crc32(tag)
    crc = zlib.crc32(data, crc)
    crc &= 0xffffffff
    crc_as_u4 = np.array([crc], dtype=">u4")
    out[-4:] = crc_as_u4.view("u1")

    return out

The three chunks are `"IHDR"` specifying a width, height, and `8, 6, 0, 0, 0` for "color image," followed by `"IDAT"` with compressed image data, followed by `"IEND"` with nothing.

In [ ]:
def png_image(imagedata):
    height, scanline_width = imagedata.shape
    width = int((scanline_width - 1) / 4)
    
    width_height = np.array([width, height], dtype=">u4")
    color_image = np.array([8, 6, 0, 0, 0], dtype="u1")
    headerdata = np.concatenate([width_height.view("u1"), color_image])
    
    preamble = png_preamble()
    header = png_chunk(b"IHDR", headerdata.tostring())
    data = png_chunk(b"IDAT", zlib.compress(imagedata.tostring()))
    end = png_chunk(b"IEND", b"")
    
    return np.concatenate([preamble, header, data, end])

Now let's try it out on some randomly generated data.

In [ ]:
width, height = 400, 300
imagedata = np.zeros((height, 4 * width + 1), dtype="u1")
imagedata[:, 1:] = np.random.randint(0, 256, (height, 4 * width))
imagedata

In [ ]:
png_image(imagedata)

In [ ]:
import IPython.display

def png_display(imagedata):
    return IPython.display.display(IPython.display.Image(data=png_image(imagedata)))

def png_save(filename, imagedata):
    with open(filename, "wb") as file:
        file.write(png_image(imagedata))   # a real PNG file that you can view with other programs

png_display(imagedata)

(You may be itching to tell me that Matplotlib's `imshow` does that, but the DIY solution will pay off in pedagogy.)

The part we skipped over was how to construct the image itself. We saw that it was an array with shape `(height, 4 * width + 1)` and the first byte of each row ("scanline") was zero. Here's a figure from the specification:

<center><img src="img/png-spec-scanline.png" width="60%"></center>

After the "filter type" (zero is simplest), they're alternating red, green, blue, "alpha" (transparency) bytes. If, for instance, we want to make the image all red, we'd set the red bytes and the alpha bytes to maximum (255), the others to zero. Use slices!

In [ ]:
imagedata = np.zeros((height, 4 * width + 1), dtype="u1")
imagedata[:, 1::4] = 255     # max out red bytes
imagedata[:, 4::4] = 255     # max out alpha bytes
png_display(imagedata)

**Exercise:** Make a gradient from black to red using `linspace`:

In [ ]:
np.linspace(0, 255, 20, dtype="u1")

In [ ]:
???

**Harder exercise:** Now use multiplication by a `linspace` to make the gradient peak in the bottom-right corner.

In [ ]:
???

For flair, let's put a streak of blue through it.

In [ ]:
imagedata[:, 3::4] = np.linspace(255, 0, width, dtype="u1")
png_display(imagedata)

We can chop up sections and move them around.

In [ ]:
imagedata[100:250, 601:681] = imagedata[100:250, 1201:1281]
png_display(imagedata)

And `roll` sections, like we did with the Rubik's square.

In [ ]:
imagedata[:, 801:881] = np.roll(imagedata[:, 801:881], 100, axis=0)
png_display(imagedata)

In [ ]:
imagedata[230:250, 1:] = np.roll(imagedata[230:250, 1:], 400, axis=1)
png_display(imagedata)

I'd like to draw your attention to that `axis` parameter. When we applied `roll` to the 2D array, we could tell it which dimension to roll. This is a general feature of Numpy:

   * `axis=0` means apply it to the first axis (the first dimension you'd slice in square brackets)
   * `axis=1` means apply it to the second
   * and so on.

Numpy arrays are truly N-dimensional objects. Nowadays, they'd be called "tensors" to sound cool.

Speaking of tensors, Numpy has a linear algebra module (usually compiles to BLAS/LAPACK, but can be ATLAS or MKL).

In [ ]:
matrix = np.array([[1.1, 2.2], [3.3, 4.4]])
matrix

In [ ]:
matrix.T    # shortcut for .transpose()

In [ ]:
np.linalg.inv(matrix)

In [ ]:
np.eye(2)   # identity or "I" (get it?)

In [ ]:
matrix @ matrix   # "at" sign means matrix multiplication in Python 3.5 and above

In [ ]:
np.matmul(matrix, matrix)   # if you're still living in Python 2.7

In [ ]:
np.linalg.matrix_power(matrix, 2)

In [ ]:
for eigenvector, eigenvalue in zip(np.linalg.eig(matrix), np.linalg.eigvals(matrix)):
    print("")
    print(eigenvector)
    print(eigenvalue)

In [ ]:
np.linalg.solve(matrix, eigenvector)

And many others. See the documentation.

Numpy has some basic statistics functions as well.

In [ ]:
a = np.random.normal(100, 5, 10000)
b = np.random.normal(0, 2, 10000) + a

In [ ]:
np.min(a), np.max(a)      # use nanmin, nanmax to ignore NaN

In [ ]:
np.mean(a)

In [ ]:
np.median(a)

In [ ]:
np.std(a)

In [ ]:
np.corrcoef(a, b)

In [ ]:
np.histogram(a, bins=10, range=(75, 125))    # counts in each bin, bin edges

**Useful odds and ends:** In my very first example, I used `where` as an if-then-else:

In [ ]:
np.where(
    np.isnan(temperatures),# condition
    0.5 * (min_temperatures + max_temperatures),# if true
         temperatures# if false
)

It can also be used to find the indexes of an array where the condition is non-zero.

In [ ]:
np.where(np.isnan(temperatures))

These indexes can then be used as a slice, much like the output of `argsort`, `argwhere`, etc. The reason it's a 1-tuple (notice the comma at the end?) is because it would give you N arrays for an N dimensional index.

In [ ]:
np.where(imagedata > 250)

In [ ]:
imagedata[np.where(imagedata > 250)]

**Here's another goodie:** sometimes you want a discrete lookup table. If the table has size `m` and the data you need to look up has size `n`, a naive search would take `O(n*m)` time. But if the lookup table is sorted, it only has to be `O(n*log(m))`.

In [ ]:
independent_variable = np.array([1.1, 1000.0, 2013.0216, 2099.99, 9999.9, 1e12, 2e55])   # size m
dependent_variable = np.array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7])                       # size m

dataset = np.exp(np.random.normal(0, 3, 100000)**2)                                   # size n
dataset

In [ ]:
indexes = np.searchsorted(independent_variable, dataset, side="left")       # bisection search every element in dataset

indexes[indexes == len(independent_variable)] = len(independent_variable) - 1  # clamp outliers to the valid range

dependent_variable[indexes]

(This is a low-level implementation of a sparse matrix...)

**Another gem:** Cumulative sum.

In [ ]:
random_numbers = np.random.normal(0, 1, 1000)

In [ ]:
random_walk = np.cumsum(random_numbers)
random_walk

**Splitting, stacking, and concatenating:** Numpy arrays must be contiguous in memory. Sometimes you need to work on a chunk at a time and then combine results.

In [ ]:
a = np.arange(103)
abits = np.array_split(a, 10)
abits

In [ ]:
np.concatenate(abits)

In [ ]:
b = np.arange(100)
bbits = np.split(b, 10)    # must be exactly the same size
bbits

In [ ]:
np.vstack(bbits)    # hstack, vstack, dstack...

**Universal functions:** We've been using "universal functions" or "ufuncs" all along now: they're functions of n numbers → 1 number for small n, such as addition, trigonometry, exponentiation...

In [ ]:
a = np.arange(10) * 1.1

In [ ]:
a + a

In [ ]:
np.add(a, a)    # same thing

In [ ]:
np.sin(a)

In [ ]:
np.exp(a)

In [ ]:
type(np.add), type(np.sin), type(np.exp)

This function type constitutes a special protocol: some third-party libraries like SciPy define many more functions in this form and other third-party libraries override them with special powers.

Numpy ufuncs standardize fast computation just as Numpy arrays standardize views of bytes in memory.

They also have some "hidden" features:

In [ ]:
[x for x in dir(np.add) if not x.startswith("_")]

In [ ]:
a = np.array([1, 2, 3, 4, 5])
np.add.reduce(a)

In [ ]:
np.sum(a)

Okay, same result, but the `ufunc.reduce` gives you a reducer for any two-argument operation:

In [ ]:
np.multiply.reduce(a)

In [ ]:
np.bitwise_or.reduce(a)

Add (or whatever) irregularly sized subsequences of an array with `ufunc.reduceat`.

In [ ]:
a = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
i = np.array([0, 3, 5, 6, 9])
np.add.reduceat(a, i)

In [ ]:
(1 + 2), (3 + 4), (5), (6 + 7 + 8), (9)

Outer products (or sums or whatever) with `ufunc.outer`.

In [ ]:
a = np.arange(10)
b = np.arange(10) * 0.1

In [ ]:
np.add.outer(a, b)

`ufunc.at` is like assignment with fancy indexing, except that repeated values are accumulated.

In [ ]:
random_integers = np.random.randint(0, 10, 15)
random_integers

In [ ]:
not_a_histogram = np.zeros(10, dtype=np.int64)
not_a_histogram[random_integers] += 1
not_a_histogram

In [ ]:
yes_a_histogram = np.zeros(10, dtype=np.int64)
np.add.at(yes_a_histogram, random_integers, 1)
yes_a_histogram

**Saving your work:** Numpy has a file format; actually just raw data with `dtype/shape` interpretations in ZIP files.

In [ ]:
np.savez("output.npz", one=a, two=imagedata)

In [ ]:
file = np.load("output.npz")

In [ ]:
file["one"]

In [ ]:
file["two"]

This format is simple but pretty efficient. (Use `numpy.savez_compressed` to also compress the arrays.)

For more options and interoperability, use the h5py or pytables libraries to save/load HDF5.

This is a good place to stop because we'll cover other libraries in the Numpy ecosystem after lunch.